# Constructing, running, and post-processing a MODFLOW groundwater model

FloPy is a popular Python package for constructing, running, and post-processing MODFLOW models.  In this example, we will demonstrate how FloPy can be used to create a numerical model of groundwater extraction.

We will work through the following steps as part of this notebook:
* Step 1. Copy and paste the getting started example from the [FloPy Website](https://github.com/modflowpy/flopy/tree/develop?tab=readme-ov-file#getting-started)
* Step 2. Modify the example to simulate steady-state groundwater extraction from a single well in the center of the domain
* Step 3. Modify the example for transient conditions
* Step 4. Modify the example to include multiple groundwater extraction wells

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import flopy

## Step 1. Run Flopy Getting Started example

Copy the MODFLOW 6 quickstart example from the [FloPy Website](https://github.com/modflowpy/flopy/tree/develop?tab=readme-ov-file#getting-started) and make sure that it runs.  Note that you will need MODFLOW in your path in order for the example to run properly.

In [1]:
# copy quickstart example and make sure it works

## Step 2. Modify example for steady-state groundwater extraction

There are several things we will need to do in order to modify the FloPy quickstart example to represent our groundwater extraction problem.  These include:

1.  Modify the Discretization (DIS) Package to properly set nlay, nrow, ncol, delr, delc, top, and botm.
2.  Modify the Node Property Flow (NPF) Package to properly set the hydraulic conductivity (named `k`).
3.  Modify the Constant Head (CHD) Package to assign constant heads around the perimeter of the model with a value of 10.
4.  Modify the Well (WEL) Package to assign an extraction well in the middle of the model domain.

Let's set the following variables to help us modify this example

```
# dis
length_x = 150000
length_y = 150000
nrow = 21
ncol = 21
delr = length_x / ncol
delc = length_y / nrow
nlay = 1
top = 10.
botm = 0.

# npf
hydraulic_conductivity = 100.

# chd
ibound = np.empty((nlay, nrow, ncol), dtype=int)
ibound.fill(-1)
ibound[:, 1:nrow-1, 1:ncol-1] = 1
layers, rows, columns = np.where(ibound == -1)
chd_spd = [[l, r, c, 10.] for l, r, c in zip(layers, rows, columns)]

# wel
well_row = int((nrow - 1) / 2)
well_column = int((ncol - 1) / 2)
well_q = -1000.
wel_spd = [[0, well_row, well_column, well_q]]
```

In [2]:
# set the dis, npf, chd, and wel variables

In [3]:
# copy the quickstart example and update the dis, npf, chd, and wel variables

## Step 3. Modify the example for transient conditions

To change the steady state model from Step 2 into a transient model, we need to:
1.  Change information in the Temporal Discretization (TDIS) Package
2.  Assign a proper starting head to the Initial Conditions (IC) Package

```
# TDIS Package
nper = 1
nstp = 100
perlen = 100.
tsmult = 1.
perioddata = [(perlen, nstp, tsmult)]

# IC Package
strt = 10.
```

In [4]:
# set the tdis and ic variables

In [5]:
# copy the script from step 2 and update the tdis and ic variables

In [6]:
# make a plot of drawdown versus time for the model cell with the well

## Step 4. Transient simulation with multiple pumping wells

Using the following `well_list`, create a well stress period variable.

In [8]:
well_text = """WELL1 49988.2 40903.66 605.0
WELL2 42195.49 5996.99 12.0
WELL3 14583.68 15884.9 716.0
WELL4 34448.56 27964.24 334.0
WELL5 22419.85 31224.71 100.0
WELL6 32417.15 4822.61 439.0
WELL7 16320.24 13385.98 694.0
WELL8 45323.84 36436.13 651.0
WELL9 28248.69 39668.15 558.0
WELL10 11045.92 31436.03 418.0
WELL11 10566.4 8672.29 730.0
WELL12 695.16 33597.84 268.0
WELL13 9036.03 2583.99 312.0
WELL14 44124.26 35123.48 483.0
WELL15 22434.9 35106.7 845.0
WELL16 22566.33 33533.98 506.0
WELL17 2285.95 1383.14 62.0
"""

# Create the list of wells (offset by 50000)
well_list = []
for w in well_text.splitlines():
    w = w.split()
    t = (w[0], float(w[1]) + 50000., float(w[2]) + 50000, float(w[3]))
    well_list.append(t)
well_list

[('WELL1', 99988.2, 90903.66, 605.0),
 ('WELL2', 92195.48999999999, 55996.99, 12.0),
 ('WELL3', 64583.68, 65884.9, 716.0),
 ('WELL4', 84448.56, 77964.24, 334.0),
 ('WELL5', 72419.85, 81224.70999999999, 100.0),
 ('WELL6', 82417.15, 54822.61, 439.0),
 ('WELL7', 66320.24, 63385.979999999996, 694.0),
 ('WELL8', 95323.84, 86436.13, 651.0),
 ('WELL9', 78248.69, 89668.15, 558.0),
 ('WELL10', 61045.92, 81436.03, 418.0),
 ('WELL11', 60566.4, 58672.29, 730.0),
 ('WELL12', 50695.16, 83597.84, 268.0),
 ('WELL13', 59036.03, 52583.99, 312.0),
 ('WELL14', 94124.26000000001, 85123.48000000001, 483.0),
 ('WELL15', 72434.9, 85106.7, 845.0),
 ('WELL16', 72566.33, 83533.98000000001, 506.0),
 ('WELL17', 52285.95, 51383.14, 62.0)]

In [9]:
# Create well stress period data by interesting each well with modelgrid
wel_spd = []
for w in well_list:
    wx = w[1]
    wy = w[2]
    w_i, w_j = gwf.modelgrid.intersect(wx, wy)
    wt = (0, w_i, w_j, -w[3])
    wel_spd.append(wt)
wel_spd

[(0, 8, 13, -605.0),
 (0, 13, 12, -12.0),
 (0, 11, 9, -716.0),
 (0, 10, 11, -334.0),
 (0, 9, 10, -100.0),
 (0, 13, 11, -439.0),
 (0, 12, 9, -694.0),
 (0, 8, 13, -651.0),
 (0, 8, 10, -558.0),
 (0, 9, 8, -418.0),
 (0, 12, 8, -730.0),
 (0, 9, 7, -268.0),
 (0, 13, 8, -312.0),
 (0, 9, 13, -483.0),
 (0, 9, 10, -845.0),
 (0, 9, 10, -506.0),
 (0, 13, 7, -62.0)]

In [7]:
# copy the script from step 3 and simulate extraction from multiple wells